### 5.5) Gathering Input Graphs

In [1]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.5.1+cu124.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.5.1+cu124.html
!pip install torch-geometric

Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 88.8 MB/s eta 0:00:00:00:01:01
Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 41.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.0 MB/s eta 0:00:00a 0:00:01


In [2]:
import torch
from torch_geometric.data import Data
from torch.serialization import add_safe_globals
import os

# Allow PyTorch to unpickle torch_geometric.data.Data objects
add_safe_globals([Data])

# Then load the files
graph_dir = "/kaggle/input/chicken-productivity-rate-graphs/graph_data_zip"
pt_files = [f for f in os.listdir(graph_dir) if f.endswith('.pt')]

graphs = [
    torch.load(os.path.join(graph_dir, f), weights_only=False)
    for f in pt_files
]

/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_scatter/_scatter_cuda.so: undefined symbol: _ZN2at4_ops16div__Tensor_mode4callERNS_6TensorERKS2_St8optionalIN3c1017basic_string_viewIcEEE
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_sparse/_spmm_cuda.so: undefined symbol: _ZN5torch8autograd12VariableInfoC1ERKN2at6TensorE
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [3]:
graphs

[Data(x=[560, 560], edge_index=[2, 440258], edge_attr=[440258, 4], y=[1]),
 Data(x=[603, 603], edge_index=[2, 319768], edge_attr=[319768, 4], y=[1]),
 Data(x=[525, 525], edge_index=[2, 92854], edge_attr=[92854, 4], y=[1]),
 Data(x=[648, 648], edge_index=[2, 477458], edge_attr=[477458, 4], y=[1]),
 Data(x=[484, 484], edge_index=[2, 327346], edge_attr=[327346, 4], y=[1]),
 Data(x=[430, 430], edge_index=[2, 295144], edge_attr=[295144, 4], y=[1]),
 Data(x=[716, 716], edge_index=[2, 348140], edge_attr=[348140, 4], y=[1]),
 Data(x=[706, 706], edge_index=[2, 401906], edge_attr=[401906, 4], y=[1]),
 Data(x=[577, 577], edge_index=[2, 488282], edge_attr=[488282, 4], y=[1]),
 Data(x=[599, 599], edge_index=[2, 582338], edge_attr=[582338, 4], y=[1]),
 Data(x=[451, 451], edge_index=[2, 274074], edge_attr=[274074, 4], y=[1]),
 Data(x=[426, 426], edge_index=[2, 277242], edge_attr=[277242, 4], y=[1]),
 Data(x=[591, 591], edge_index=[2, 571268], edge_attr=[571268, 4], y=[1]),
 Data(x=[199, 199], edge_in

In [4]:
# padding so model can expect a fixed set of dimensions from graphs

max_node_dim = max(g.x.shape[1] for g in graphs)
max_edge_dim = max(g.edge_attr.shape[1] for g in graphs)

def pad_features(graph, max_node_dim, max_edge_dim):
    # Pad node features
    node_feat = graph.x
    if node_feat.shape[1] < max_node_dim:
        pad_size = max_node_dim - node_feat.shape[1]
        padding = torch.zeros((node_feat.shape[0], pad_size), dtype=node_feat.dtype)
        graph.x = torch.cat([node_feat, padding], dim=1)
    
    # Pad edge features
    edge_feat = graph.edge_attr
    if edge_feat.shape[1] < max_edge_dim:
        pad_size = max_edge_dim - edge_feat.shape[1]
        padding = torch.zeros((edge_feat.shape[0], pad_size), dtype=edge_feat.dtype)
        graph.edge_attr = torch.cat([edge_feat, padding], dim=1)
    
    return graph

In [5]:
graphs_padded = [pad_features(g, max_node_dim, max_edge_dim) for g in graphs]

### 6) Create GAT Model for Graph Regression¶

GAT model takes graph as input, applies GAT layers to learn node embeddings, aggregates (mean pooling or attention pooling) to a graph level embedding, and passes that to fully connected layers to predict a single value (productivity)

global mean pooling

In [50]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv, global_mean_pool
from torch_geometric.loader import DataLoader
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

class GATGraphRegressor(torch.nn.Module):
    def __init__(self, in_node_feats, hidden_dim=64, heads=4):
        super(GATGraphRegressor, self).__init__()
        self.gat1 = GATConv(in_node_feats, hidden_dim, heads=heads, concat=True, dropout=0.2)
        self.gat2 = GATConv(hidden_dim * heads, hidden_dim, heads=1, concat=True, dropout=0.2)

        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, 1)
        )

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.gat1(x, edge_index)
        x = F.elu(x)
        x = self.gat2(x, edge_index)
        x = F.elu(x)
        x = global_mean_pool(x, batch)
        return self.mlp(x).squeeze(1)

def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    losses = []
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch)
        loss = criterion(out, batch.y.view(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        losses.append(loss.item())
    return np.mean(losses)

def eval_model(model, loader, criterion, device):
    model.eval()
    losses = []
    preds, targets = [], []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            out = model(batch)
            loss = criterion(out, batch.y.view(-1))
            losses.append(loss.item())
            preds.extend(out.cpu().numpy())
            targets.extend(batch.y.view(-1).cpu().numpy())
    rmse = mean_squared_error(targets, preds, squared=False)
    mae = mean_absolute_error(targets, preds)
    return np.mean(losses), rmse, mae

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
kf = KFold(n_splits=5, shuffle=True, random_state=0)

all_rmse = []
all_mae = []

for fold, (train_idx, val_idx) in enumerate(kf.split(graphs_padded)):
    print(f"Fold {fold + 1}")
    train_dataset = [graphs_padded[i] for i in train_idx]
    val_dataset = [graphs_padded[i] for i in val_idx]

    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

    model = GATGraphRegressor(in_node_feats=max_node_dim).to(device)
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    best_val_loss = float('inf')
    patience = 10
    patience_counter = 0

    for epoch in range(100):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, rmse, mae = eval_model(model, val_loader, criterion, device)
        print(f"Epoch {epoch} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | RMSE: {rmse:.4f} | MAE: {mae:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), f"best_model_fold{fold}.pt")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping")
                break

    # Load best model and evaluate on validation set
    model.load_state_dict(torch.load(f"best_model_fold{fold}.pt"))
    _, rmse, mae = eval_model(model, val_loader, criterion, device)
    all_rmse.append(rmse)
    all_mae.append(mae)
    print(f"Fold {fold + 1} final RMSE: {rmse:.4f}, MAE: {mae:.4f}")

print(f"\nAverage RMSE: {np.mean(all_rmse):.4f} ± {np.std(all_rmse):.4f}")
print(f"Average MAE: {np.mean(all_mae):.4f} ± {np.std(all_mae):.4f}")


Fold 1
Epoch 0 | Train Loss: 1.0966 | Val Loss: 0.7271 | RMSE: 0.8527 | MAE: 0.7572
Epoch 1 | Train Loss: 1.0875 | Val Loss: 0.7656 | RMSE: 0.8750 | MAE: 0.7719
Epoch 2 | Train Loss: 1.0909 | Val Loss: 0.8673 | RMSE: 0.9313 | MAE: 0.8030
Epoch 3 | Train Loss: 1.2166 | Val Loss: 0.9918 | RMSE: 0.9959 | MAE: 0.8392
Epoch 4 | Train Loss: 1.1289 | Val Loss: 0.9736 | RMSE: 0.9867 | MAE: 0.8255
Epoch 5 | Train Loss: 1.1602 | Val Loss: 0.9578 | RMSE: 0.9787 | MAE: 0.8086
Epoch 6 | Train Loss: 1.1021 | Val Loss: 0.8801 | RMSE: 0.9382 | MAE: 0.7701
Epoch 7 | Train Loss: 1.0085 | Val Loss: 0.7621 | RMSE: 0.8730 | MAE: 0.7102
Epoch 8 | Train Loss: 0.9108 | Val Loss: 0.5686 | RMSE: 0.7541 | MAE: 0.6253
Epoch 9 | Train Loss: 0.8200 | Val Loss: 0.4713 | RMSE: 0.6865 | MAE: 0.5622
Epoch 10 | Train Loss: 0.7599 | Val Loss: 0.3812 | RMSE: 0.6174 | MAE: 0.5101
Epoch 11 | Train Loss: 0.6646 | Val Loss: 0.2838 | RMSE: 0.5328 | MAE: 0.4719
Epoch 12 | Train Loss: 0.5779 | Val Loss: 0.2086 | RMSE: 0.4568 | M

global_max_pool instead of global_mean_pool

In [51]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv, global_max_pool
from torch_geometric.loader import DataLoader
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

class GATGraphRegressor(torch.nn.Module):
    def __init__(self, in_node_feats, hidden_dim=64, heads=4):
        super(GATGraphRegressor, self).__init__()
        self.gat1 = GATConv(in_node_feats, hidden_dim, heads=heads, concat=True, dropout=0.2)
        self.gat2 = GATConv(hidden_dim * heads, hidden_dim, heads=1, concat=True, dropout=0.2)

        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, 1)
        )

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.gat1(x, edge_index)
        x = F.elu(x)
        x = self.gat2(x, edge_index)
        x = F.elu(x)
        x = global_max_pool(x, batch)
        return self.mlp(x).squeeze(1)

# Normalize targets
ys_all = [g.y.item() for g in graphs_padded]
mean_y = np.mean(ys_all)
std_y = np.std(ys_all)

for g in graphs_padded:
    g.y = torch.tensor([(g.y.item() - mean_y) / std_y], dtype=torch.float)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

kf = KFold(n_splits=5, shuffle=True, random_state=0)

def train_epoch(model, loader, criterion, optimizer):
    model.train()
    losses = []
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch)
        loss = criterion(out, batch.y.view(-1))
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    return np.mean(losses)

def eval_model(model, loader, criterion):
    model.eval()
    losses = []
    preds, targets = [], []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            out = model(batch)
            loss = criterion(out, batch.y.view(-1))
            losses.append(loss.item())
            preds.extend(out.cpu().numpy())
            targets.extend(batch.y.view(-1).cpu().numpy())
    rmse = mean_squared_error(targets, preds, squared=False)
    mae = mean_absolute_error(targets, preds)
    return np.mean(losses), rmse, mae

all_rmse = []
all_mae = []

for fold, (train_idx, val_idx) in enumerate(kf.split(graphs_padded)):
    print(f"Fold {fold + 1}")
    train_dataset = [graphs_padded[i] for i in train_idx]
    val_dataset = [graphs_padded[i] for i in val_idx]

    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

    model = GATGraphRegressor(in_node_feats=max_node_dim).to(device)
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    best_val_loss = float('inf')
    patience = 10
    patience_counter = 0

    for epoch in range(100):
        train_loss = train_epoch(model, train_loader, criterion, optimizer)
        val_loss, rmse, mae = eval_model(model, val_loader, criterion)
        print(f"Epoch {epoch} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | RMSE: {rmse:.4f} | MAE: {mae:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), f"best_model_max_fold{fold}.pt")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break

    model.load_state_dict(torch.load(f"best_model_max_fold{fold}.pt"))
    _, rmse, mae = eval_model(model, val_loader, criterion)
    all_rmse.append(rmse)
    all_mae.append(mae)
    print(f"Fold {fold + 1} final RMSE: {rmse:.4f}, MAE: {mae:.4f}")

print(f"\nAverage RMSE: {np.mean(all_rmse):.4f} ± {np.std(all_rmse):.4f}")
print(f"Average MAE: {np.mean(all_mae):.4f} ± {np.std(all_mae):.4f}")

Fold 1


/usr/local/lib/python3.11/dist-packages/torch_geometric/warnings.py:11: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(message)


Epoch 0 | Train Loss: 1.0928 | Val Loss: 0.6418 | RMSE: 0.8011 | MAE: 0.7292


/usr/local/lib/python3.11/dist-packages/torch_geometric/warnings.py:11: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(message)


Epoch 1 | Train Loss: 1.0855 | Val Loss: 0.6413 | RMSE: 0.8008 | MAE: 0.7277


...

Epoch 97 | Train Loss: 0.0278 | Val Loss: 0.3933 | RMSE: 0.6272 | MAE: 0.4433


/usr/local/lib/python3.11/dist-packages/torch_geometric/warnings.py:11: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(message)


Epoch 98 | Train Loss: 0.0294 | Val Loss: 0.3954 | RMSE: 0.6288 | MAE: 0.4640


/usr/local/lib/python3.11/dist-packages/torch_geometric/warnings.py:11: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(message)


Epoch 99 | Train Loss: 0.0377 | Val Loss: 0.4389 | RMSE: 0.6625 | MAE: 0.5200
Fold 5 final RMSE: 0.6135, MAE: 0.4340

Average RMSE: 0.5221 ± 0.1723
Average MAE: 0.3760 ± 0.0689


Use now globalAttention

In [52]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv, GlobalAttention
from torch_geometric.loader import DataLoader
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

class GATGraphRegressor(torch.nn.Module):
    def __init__(self, in_node_feats, hidden_dim=64, heads=4):
        super(GATGraphRegressor, self).__init__()

        self.gat1 = GATConv(in_node_feats, hidden_dim, heads=heads, concat=True, dropout=0.2)
        self.gat2 = GATConv(hidden_dim * heads, hidden_dim, heads=1, concat=True, dropout=0.2)
        
        self.att_pool = GlobalAttention(gate_nn=torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, 1),
            torch.nn.Sigmoid()
        ))
        
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, 1)
        )

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.gat1(x, edge_index)
        x = F.elu(x)
        x = self.gat2(x, edge_index)
        x = F.elu(x)
        x = self.att_pool(x, batch)
        return self.mlp(x).squeeze(1)

# Normalize targets
ys_all = [g.y.item() for g in graphs_padded]
mean_y = np.mean(ys_all)
std_y = np.std(ys_all)

for g in graphs_padded:
    g.y = torch.tensor([(g.y.item() - mean_y) / std_y], dtype=torch.float)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

kf = KFold(n_splits=5, shuffle=True, random_state=0)

def train_epoch(model, loader, criterion, optimizer):
    model.train()
    losses = []
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch)
        loss = criterion(out, batch.y.view(-1))
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    return np.mean(losses)

def eval_model(model, loader, criterion):
    model.eval()
    losses = []
    preds, targets = [], []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            out = model(batch)
            loss = criterion(out, batch.y.view(-1))
            losses.append(loss.item())
            preds.extend(out.cpu().numpy())
            targets.extend(batch.y.view(-1).cpu().numpy())
    rmse = mean_squared_error(targets, preds, squared=False)
    mae = mean_absolute_error(targets, preds)
    return np.mean(losses), rmse, mae

all_rmse = []
all_mae = []

for fold, (train_idx, val_idx) in enumerate(kf.split(graphs_padded)):
    print(f"Fold {fold + 1}")
    train_dataset = [graphs_padded[i] for i in train_idx]
    val_dataset = [graphs_padded[i] for i in val_idx]

    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

    model = GATGraphRegressor(in_node_feats=max_node_dim).to(device)
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    best_val_loss = float('inf')
    patience = 10
    patience_counter = 0

    for epoch in range(100):
        train_loss = train_epoch(model, train_loader, criterion, optimizer)
        val_loss, rmse, mae = eval_model(model, val_loader, criterion)
        print(f"Epoch {epoch} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | RMSE: {rmse:.4f} | MAE: {mae:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), f"best_model_ga_fold{fold}.pt")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break

    model.load_state_dict(torch.load(f"best_model_ga_fold{fold}.pt"))
    _, rmse, mae = eval_model(model, val_loader, criterion)
    all_rmse.append(rmse)
    all_mae.append(mae)
    print(f"Fold {fold + 1} final RMSE: {rmse:.4f}, MAE: {mae:.4f}")

print(f"\nAverage RMSE: {np.mean(all_rmse):.4f} ± {np.std(all_rmse):.4f}")
print(f"Average MAE: {np.mean(all_mae):.4f} ± {np.std(all_mae):.4f}")

Fold 1


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'nn.glob.GlobalAttention' is deprecated, use 'nn.aggr.AttentionalAggregation' instead
  warnings.warn(out)


Epoch 0 | Train Loss: 1.0948 | Val Loss: 0.6307 | RMSE: 0.7942 | MAE: 0.7261
Epoch 1 | Train Loss: 1.0821 | Val Loss: 0.6284 | RMSE: 0.7927 | MAE: 0.7219
Epoch 2 | Train Loss: 1.0476 | Val Loss: 0.6133 | RMSE: 0.7831 | MAE: 0.7084
Epoch 3 | Train Loss: 0.9792 | Val Loss: 0.5307 | RMSE: 0.7285 | MAE: 0.6632
Epoch 4 | Train Loss: 0.8630 | Val Loss: 0.4420 | RMSE: 0.6648 | MAE: 0.5995
Epoch 5 | Train Loss: 0.7123 | Val Loss: 0.2898 | RMSE: 0.5383 | MAE: 0.4973
Epoch 6 | Train Loss: 0.5579 | Val Loss: 0.1870 | RMSE: 0.4324 | MAE: 0.3685
Epoch 7 | Train Loss: 0.4529 | Val Loss: 0.1568 | RMSE: 0.3960 | MAE: 0.3044
Epoch 8 | Train Loss: 0.4025 | Val Loss: 0.1599 | RMSE: 0.3999 | MAE: 0.2898
Epoch 9 | Train Loss: 0.3796 | Val Loss: 0.1585 | RMSE: 0.3981 | MAE: 0.2843
Epoch 10 | Train Loss: 0.3622 | Val Loss: 0.1745 | RMSE: 0.4177 | MAE: 0.2946
Epoch 11 | Train Loss: 0.3565 | Val Loss: 0.2215 | RMSE: 0.4706 | MAE: 0.3531
Epoch 12 | Train Loss: 0.3535 | Val Loss: 0.2002 | RMSE: 0.4475 | MAE: 0.3

/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'nn.glob.GlobalAttention' is deprecated, use 'nn.aggr.AttentionalAggregation' instead
  warnings.warn(out)


Epoch 0 | Train Loss: 1.1484 | Val Loss: 0.4295 | RMSE: 0.6554 | MAE: 0.6155
Epoch 1 | Train Loss: 1.1327 | Val Loss: 0.4385 | RMSE: 0.6622 | MAE: 0.6235
Epoch 2 | Train Loss: 1.1017 | Val Loss: 0.4347 | RMSE: 0.6593 | MAE: 0.6220
Epoch 3 | Train Loss: 1.0455 | Val Loss: 0.3961 | RMSE: 0.6293 | MAE: 0.5919
Epoch 4 | Train Loss: 0.9565 | Val Loss: 0.3074 | RMSE: 0.5544 | MAE: 0.5152
Epoch 5 | Train Loss: 0.8137 | Val Loss: 0.1977 | RMSE: 0.4446 | MAE: 0.3965
Epoch 6 | Train Loss: 0.6491 | Val Loss: 0.1041 | RMSE: 0.3227 | MAE: 0.2675
Epoch 7 | Train Loss: 0.5038 | Val Loss: 0.0804 | RMSE: 0.2836 | MAE: 0.2203
Epoch 8 | Train Loss: 0.4274 | Val Loss: 0.0794 | RMSE: 0.2818 | MAE: 0.1983
Epoch 9 | Train Loss: 0.4015 | Val Loss: 0.1186 | RMSE: 0.3443 | MAE: 0.2306
Epoch 10 | Train Loss: 0.3935 | Val Loss: 0.0960 | RMSE: 0.3098 | MAE: 0.2257
Epoch 11 | Train Loss: 0.3717 | Val Loss: 0.1539 | RMSE: 0.3923 | MAE: 0.2670
Epoch 12 | Train Loss: 0.3785 | Val Loss: 0.1299 | RMSE: 0.3604 | MAE: 0.2

...

Epoch 0 | Train Loss: 0.8351 | Val Loss: 1.6825 | RMSE: 1.2971 | MAE: 0.9423
Epoch 1 | Train Loss: 0.8196 | Val Loss: 1.6763 | RMSE: 1.2947 | MAE: 0.9312
Epoch 2 | Train Loss: 0.7915 | Val Loss: 1.6388 | RMSE: 1.2801 | MAE: 0.9105
Epoch 3 | Train Loss: 0.7375 | Val Loss: 1.5341 | RMSE: 1.2386 | MAE: 0.8647
Epoch 4 | Train Loss: 0.6371 | Val Loss: 1.3809 | RMSE: 1.1751 | MAE: 0.7827
Epoch 5 | Train Loss: 0.5038 | Val Loss: 1.1896 | RMSE: 1.0907 | MAE: 0.6544
Epoch 6 | Train Loss: 0.3781 | Val Loss: 1.0053 | RMSE: 1.0027 | MAE: 0.5590
Epoch 7 | Train Loss: 0.2997 | Val Loss: 0.8800 | RMSE: 0.9381 | MAE: 0.4863
Epoch 8 | Train Loss: 0.2634 | Val Loss: 0.8137 | RMSE: 0.9020 | MAE: 0.4637
Epoch 9 | Train Loss: 0.2486 | Val Loss: 0.7879 | RMSE: 0.8876 | MAE: 0.4520
Epoch 10 | Train Loss: 0.2363 | Val Loss: 0.7575 | RMSE: 0.8703 | MAE: 0.4503
Epoch 11 | Train Loss: 0.2339 | Val Loss: 0.7476 | RMSE: 0.8646 | MAE: 0.4531
Epoch 12 | Train Loss: 0.2281 | Val Loss: 0.7469 | RMSE: 0.8642 | MAE: 0.4

/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'nn.glob.GlobalAttention' is deprecated, use 'nn.aggr.AttentionalAggregation' instead
  warnings.warn(out)


Epoch 0 | Train Loss: 1.1113 | Val Loss: 0.5601 | RMSE: 0.7484 | MAE: 0.6712
Epoch 1 | Train Loss: 1.1033 | Val Loss: 0.5457 | RMSE: 0.7387 | MAE: 0.6639
Epoch 2 | Train Loss: 1.0756 | Val Loss: 0.5108 | RMSE: 0.7147 | MAE: 0.6432
Epoch 3 | Train Loss: 1.0010 | Val Loss: 0.4405 | RMSE: 0.6637 | MAE: 0.5955
Epoch 4 | Train Loss: 0.8823 | Val Loss: 0.3358 | RMSE: 0.5795 | MAE: 0.5029
Epoch 5 | Train Loss: 0.7207 | Val Loss: 0.2478 | RMSE: 0.4978 | MAE: 0.4205
Epoch 6 | Train Loss: 0.5620 | Val Loss: 0.1968 | RMSE: 0.4436 | MAE: 0.3257
Epoch 7 | Train Loss: 0.4541 | Val Loss: 0.2102 | RMSE: 0.4584 | MAE: 0.3289
Epoch 8 | Train Loss: 0.4033 | Val Loss: 0.2187 | RMSE: 0.4677 | MAE: 0.3413
Epoch 9 | Train Loss: 0.3781 | Val Loss: 0.2444 | RMSE: 0.4943 | MAE: 0.3838
Epoch 10 | Train Loss: 0.3686 | Val Loss: 0.2418 | RMSE: 0.4917 | MAE: 0.3714
Epoch 11 | Train Loss: 0.3586 | Val Loss: 0.2683 | RMSE: 0.5180 | MAE: 0.4040
Epoch 12 | Train Loss: 0.3510 | Val Loss: 0.2395 | RMSE: 0.4894 | MAE: 0.3

/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'nn.glob.GlobalAttention' is deprecated, use 'nn.aggr.AttentionalAggregation' instead
  warnings.warn(out)


Epoch 0 | Train Loss: 0.8327 | Val Loss: 1.6747 | RMSE: 1.2941 | MAE: 1.0480
Epoch 1 | Train Loss: 0.8241 | Val Loss: 1.6511 | RMSE: 1.2849 | MAE: 1.0352
Epoch 2 | Train Loss: 0.7955 | Val Loss: 1.5635 | RMSE: 1.2504 | MAE: 0.9966
Epoch 3 | Train Loss: 0.7065 | Val Loss: 1.3685 | RMSE: 1.1698 | MAE: 0.9050
Epoch 4 | Train Loss: 0.5747 | Val Loss: 1.1355 | RMSE: 1.0656 | MAE: 0.7705
Epoch 5 | Train Loss: 0.4565 | Val Loss: 0.9046 | RMSE: 0.9511 | MAE: 0.6409
Epoch 6 | Train Loss: 0.3653 | Val Loss: 0.7360 | RMSE: 0.8579 | MAE: 0.5461
Epoch 7 | Train Loss: 0.3187 | Val Loss: 0.6508 | RMSE: 0.8067 | MAE: 0.5039
Epoch 8 | Train Loss: 0.3016 | Val Loss: 0.5940 | RMSE: 0.7707 | MAE: 0.4924
Epoch 9 | Train Loss: 0.2958 | Val Loss: 0.5909 | RMSE: 0.7687 | MAE: 0.4384
Epoch 10 | Train Loss: 0.2909 | Val Loss: 0.5642 | RMSE: 0.7512 | MAE: 0.4431
Epoch 11 | Train Loss: 0.2801 | Val Loss: 0.5463 | RMSE: 0.7391 | MAE: 0.4777
Epoch 12 | Train Loss: 0.2750 | Val Loss: 0.5516 | RMSE: 0.7427 | MAE: 0.4

### 7) Evaluate

In [41]:
import pandas as pd

target = pd.read_csv("/kaggle/input/private-hen-productivity-target-labels/production_net.csv", sep=";")
eggs_per_day_df = target[["date", "Laying.rate...."]].rename(columns={"Laying.rate....": "productivity rate"})
eggs_per_day_df['productivity rate'] = eggs_per_day_df['productivity rate'].str.replace(',', '.').astype(float)
eggs_per_day_df.head(5)

,date,productivity rate
0,2017-02-27,94.960274
1,2017-03-01,94.260145
2,2017-04-25,92.655367
3,2017-07-12,89.730444
4,2017-07-22,88.271474


In [42]:
mean_eggs = eggs_per_day_df["productivity rate"].mean()
std_eggs = eggs_per_day_df["productivity rate"].std()

print(f"Mean productivity rate: {mean_eggs:.2f}")
print(f"Standard deviation of productivity rate: {std_eggs:.2f}")

Mean productivity rate: 87.16
Standard deviation of productivity rate: 8.34


## Model Evaluation: Egg Productivity Prediction

### Baseline Statistics
- **Mean productivity rate:** 87.16  
- **Standard deviation:** 8.34  

---

### Model 1: Global Mean Pooling
- **Average RMSE:** 0.5359 ± 0.2512  
- **Average MAE:** 0.3239 ± 0.1025  
- Highest error among all models — suggests mean pooling may dilute critical node-level information.

---

### Model 2: Global Max Pooling
- **Average RMSE:** 0.5221 ± 0.1723  
- **Average MAE:** 0.3760 ± 0.0689  
- Slight improvement in RMSE over mean pooling — max pooling may capture dominant node signals, though MAE increased.

---

### Model 3: Global Attention Pooling
- **Average RMSE:** 0.4623 ± 0.1234  
- **Average MAE:** 0.3067 ± 0.0578  
- Best performance overall — attention mechanism helps identify and weight important nodes, improving both accuracy and consistency.

---

### Overall Insights
- Switching from **mean to attention pooling** significantly reduced prediction error.
- Attention pooling **outperformed both mean and max** in terms of RMSE and MAE.
- **Model 3’s MAE (0.3067)** is approximately **1/27th** of the standard deviation — indicating strong prediction performance relative to variability in the data.